### <center>__画最后一年年平均的净海表热通量Net Surface Heat Flux经向热输送Heat Transport__

In [1]:
output                 = "/home/yyq/mls/dataprocess/addwater/photo/heat"
outputB                = "/home/yyq/mls/dataprocess/addwater/photo/heat_lat"
datapath               = "/data06/yyq/mls/licom2/addwater_new/exe/"
gridpath               = "/home/yyq/mls/dataprocess/grid/basin_new.nc"

In [2]:
fileslist              = systemfunc("ls "+ datapath+"MMEAN" +"*") 
f                      = addfiles(fileslist, "r") 
f_grid                 = addfile(gridpath, "r")

In [3]:
time_last              = dimsizes(fileslist) - 1

In [4]:
lev                    = f[0]->lev1
lat                    = f[0]->lat
vs_a                   = f[(time_last - 11):time_last]->vs
ts_a                   = f[(time_last - 11):time_last]->ts
net_a                   = f[(time_last - 11):time_last]->net1
indd                   = f_grid->indd

In [5]:
time_size              = dimsizes(vs_a(:,0,0,0)) - 1
lat_size               = dimsizes(vs_a(0,0,:,0)) - 1
lon_size               = dimsizes(vs_a(0,0,0,:)) - 1 - 2
lev_size               = dimsizes(lev) - 1

In [6]:
vs                     = vs_a(:,:,:,0:lon_size)
ts                     = ts_a(:,:,:,0:lon_size)
net                    = net_a(:,:,0:lon_size)
printVarSummary(vs)

Variable: vs
Type: float
Total Size: 29808000 bytes
            7452000 values
Number of Dimensions: 4
Dimensions and sizes:	[time | 12] x [lev | 30] x [lat | 115] x [lon | 180]
Coordinates: 
            time: [1057..1068]
            lev: [-5..-5243.597]
            lat: [90..-78]
            lon: [ 0..358]
Number Of Attributes: 3
  _FillValue :	1e+35
  units :	m/s
  long_name :	meridional current

In [7]:
angle2radian= 4.0 * atan(1.0) / 180.0
r_earth = 6.371e6
dlon = 2
Cp = 4.18e3
rho0 = 1027

In [8]:
r = r_earth * cos (lat * angle2radian)
dx= r * (dlon * angle2radian)
dz = new((/lev_size/),typeof(lev))
do z = 1,lev_size
    dz(z-1) = ((lev(z) - lev(z-1))) * (-1)
end do 

In [9]:
dz_conform = new((/time_size+1,lev_size,lat_size+1,lon_size+1/),float)
dz_conform = conform(dz_conform, dz, (/1/))
dx_conform = new((/time_size+1,lev_size,lat_size+1,lon_size+1/),float)
dx_conform = conform(dx_conform, dx, (/2/))
printVarSummary(dx_conform)

Variable: dx_conform
Type: float
Total Size: 29808000 bytes
            7452000 values
Number of Dimensions: 4
Dimensions and sizes:	[12] x [30] x [115] x [180]
Coordinates: 
Number Of Attributes: 1
  _FillValue :	9.96921e+36

Integrate

In [10]:
heat = vs * ts * dz_conform * dx_conform * Cp * rho0                            
heat_integrate_lev =dim_sum_n(heat, 1)/(1e15)                                  
copy_VarMeta(vs(:,0,:,:), heat_integrate_lev)
heat_integrate_lev@long_name = "meridional heat transport"
heat_integrate_lev@units = "PW"
printVarSummary(heat_integrate_lev)

Variable: heat_integrate_lev
Type: float
Total Size: 993600 bytes
            248400 values
Number of Dimensions: 3
Dimensions and sizes:	[time | 12] x [lat | 115] x [lon | 180]
Coordinates: 
            time: [1057..1068]
            lat: [90..-78]
            lon: [ 0..358]
Number Of Attributes: 4
  units :	PW
  long_name :	meridional heat transport
  lev :	-5
  _FillValue :	1e+35

net_basin

In [11]:
net_avg               = dim_avg_n(net*1e6,0)   
copy_VarMeta(net(0,:,:), net_avg)
net_avg_lat           = dim_sum_n_Wrap(net_avg,1) 

;Pacfic
net_avg_pacfic        = net_avg
net_avg_pacfic        = where(indd.eq.4, net_avg, net_avg@_FillValue)
net_avg_lat_pacfic    = dim_sum_n_Wrap(net_avg_pacfic,1)

;Atlantic
net_avg_atlantic      = net_avg
net_avg_atlantic      = where(indd.eq.2, net_avg, net_avg@_FillValue)
net_avg_lat_atlantic  = dim_sum_n_Wrap(net_avg_atlantic,1)

;Indian
net_avg_indian        = net_avg
net_avg_indian        = where(indd.eq.3, net_avg, net_avg@_FillValue)
net_avg_lat_indian    = dim_sum_n_Wrap(net_avg_indian,1)

printVarSummary(net_avg)
printVarSummary(net_avg_lat_indian)

Variable: net_avg
Type: float
Total Size: 82800 bytes
            20700 values
Number of Dimensions: 2
Dimensions and sizes:	[lat | 115] x [lon | 180]
Coordinates: 
            lat: [90..-78]
            lon: [ 0..358]
Number Of Attributes: 4
  units :	W/m^2
  long_name :	net surface heat flux
  time :	1057
  _FillValue :	1e+35Variable: net_avg_lat_indian
Type: float
Total Size: 460 bytes
            115 values
Number of Dimensions: 1
Dimensions and sizes:	[lat | 115]
Coordinates: 
            lat: [90..-78]
Number Of Attributes: 5
  _FillValue :	1e+35
  time :	1057
  long_name :	net surface heat flux
  units :	W/m^2
  sum_op_ncl :	dim_sum_n over dimension(s): lon

heat_trans_basin

In [12]:
;all basin
heat_avg               = dim_avg_n_Wrap(heat_integrate_lev,0)                                     
heat_avg_lat           = dim_sum_n_Wrap(heat_avg,1)              
heat_avg_lat           = runave(heat_avg_lat, 9, 0)
;Pacfic
heat_avg_pacfic        = heat_avg
heat_avg_pacfic        = where(indd.eq.4, heat_avg, heat_avg@_FillValue)
heat_avg_lat_pacfic    = dim_sum_n_Wrap(heat_avg_pacfic,1)
heat_avg_lat_pacfic    = runave(heat_avg_lat_pacfic, 9, 0)
;Atlantic
heat_avg_atlantic      = heat_avg
heat_avg_atlantic      = where(indd.eq.2, heat_avg, heat_avg@_FillValue)
heat_avg_lat_atlantic  = dim_sum_n_Wrap(heat_avg_atlantic,1)
heat_avg_lat_atlantic  = runave(heat_avg_lat_atlantic, 9, 0)
;Indian
heat_avg_indian        = heat_avg
heat_avg_indian        = where(indd.eq.3, heat_avg, heat_avg@_FillValue)
heat_avg_lat_indian    = dim_sum_n_Wrap(heat_avg_indian,1)
heat_avg_lat_indian    = runave(heat_avg_lat_indian, 9, 0)

printVarSummary(heat_avg)
printVarSummary(heat_avg_lat)

Variable: heat_avg
Type: float
Total Size: 82800 bytes
            20700 values
Number of Dimensions: 2
Dimensions and sizes:	[lat | 115] x [lon | 180]
Coordinates: 
            lat: [90..-78]
            lon: [ 0..358]
Number Of Attributes: 5
  _FillValue :	1e+35
  units :	PW
  long_name :	meridional heat transport
  lev :	-5
  average_op_ncl :	dim_avg_n over dimension(s): timeVariable: heat_avg_lat
Type: float
Total Size: 460 bytes
            115 values
Number of Dimensions: 1
Dimensions and sizes:	[lat | 115]
Coordinates: 
            lat: [90..-78]
Number Of Attributes: 6
  _FillValue :	1e+35
  units :	PW
  long_name :	meridional heat transport
  lev :	-5
  average_op_ncl :	dim_avg_n over dimension(s): time
  sum_op_ncl :	dim_sum_n over dimension(s): lon

In [13]:
level1                  = new((/51/),float)  
level1                  = ispan(-25,25,1)
level1                  = level1 / 10

level2                  = new((/51/),float)  
level2                  = ispan(-50,50,2)
level2                  = level2 / 10


Draw

In [14]:
wks      = gsn_open_wks("png", output)
plots    = new(2,"graphic")
res_0    = True
    res_0@gsnDraw                   =  False
    res_0@gsnFrame                  =  False
    res_0@cnFillOn                  =  True
    res_0@cnInfoLabelOn             =  False
    res_0@cnLinesOn                 =  False
    res_0@cnLineLabelsOn            =  False
    res_0@lbLabelBarOn              =  False
    res_0@cnLevelSelectionMode      = "ExplicitLevels"
    res_0@cnFillPalette             = "BlRe"
    res_0@cnLevels                  = level1
    res_0@gsnLeftString             = "Meridional Heat Transport"
    res_0@gsnLeftStringFontHeightF  = 15
    res_0@gsnRightString            = "PW"
    res_0@gsnRightStringFontHeightF = 15
    res_0@mpCenterLonF              = 180
    res_0@lbLabelBarOn              = True
    res_0@lbLabelStride              = 5                 ;colorbar每5个色块一个标注
    res_0@lbLabelAutoStride          = False             ;colorbar每条等值线均标注
    res_0@lbBoxSeparatorLinesOn      = False             ;colorbar内部边框隐藏
    res_0@lbBoxLinesOn               = False             ;colorbar外边框设置为隐藏
    res_0@pmLabelBarHeightF          = 0.05              ;colorbar高度
    res_0@pmLabelBarWidthF           = 0.8               ;colorbar宽度
    res_0@pmLabelBarOrthogonalPosF   = 0.1               ;colorbar水平偏移
    res_0@lbBoxEndCapStyle           = "TriangleBothEnds"
plots(1) = gsn_csm_contour_map(wks,heat_avg, res_0)

res_1 = True
    res_1@gsnDraw                   =  False
    res_1@gsnFrame                  =  False
    res_1@cnFillOn                  =  True
    res_1@cnInfoLabelOn             =  False
    res_1@cnLinesOn                 =  False
    res_1@cnLineLabelsOn            =  False
    res_1@lbLabelBarOn              =  False
    res_1@cnFillPalette             = "BlRe"
    res_1@cnLevelSelectionMode      = "ExplicitLevels"
    res_1@cnLevels                  = level2
    res_1@mpCenterLonF              = 180
    res_1@lbLabelBarOn              = True
    res_1@lbLabelStride              = 5                 ;colorbar每5个色块一个标注
    res_1@lbLabelAutoStride          = False             ;colorbar每条等值线均标注
    res_1@lbBoxSeparatorLinesOn      = False             ;colorbar内部边框隐藏
    res_1@lbBoxLinesOn               = False             ;colorbar外边框设置为隐藏
    res_1@pmLabelBarHeightF          = 0.05              ;colorbar高度
    res_1@pmLabelBarWidthF           = 0.8               ;colorbar宽度
    res_1@pmLabelBarOrthogonalPosF   = 0.1               ;colorbar水平偏移
    res_1@lbBoxEndCapStyle           = "TriangleBothEnds"
plots(0) = gsn_csm_contour_map(wks,net_avg, res_1)

gsn_panel(wks, plots, (/1,2/), True)

.git      addwater  grid      test      tool      warming.git      addwater  grid      test      tool      warming

![heat](../photo/heat.png)

Draw lat_avg

In [15]:
y2                      = new((/4,lat_size + 1/),float)
y2(3,:)                 = heat_avg_lat
y2(2,:)                 = heat_avg_lat_pacfic
y2(1,:)                 = heat_avg_lat_atlantic
y2(0,:)                 = heat_avg_lat_indian

y1                      = new((/4,lat_size + 1/),float)
y1(3,:)                 = net_avg_lat
y1(2,:)                 = net_avg_lat_pacfic
y1(1,:)                 = net_avg_lat_atlantic
y1(0,:)                 = net_avg_lat_indian

In [16]:
wksB                              = gsn_open_wks("png", outputB)
plotsB                            = new(2,"graphic")
resB_0    = True
    resB_0@gsnDraw                = False
    resB_0@gsnFrame               = False           
    resB_0@vpHeightF              = 0.37
    resB_0@vpWidthF               = 0.6
    resB_0@tiXAxisString          = "Latitude" 
    resB_0@tiYAxisString          = "Heat Transport(PW)" 
    resB_0@gsnYRefLine            = 0
    resB_0@gsnYRefLineThicknessF  = 1.5
    resB_0@xyLineColors           = (/"purple","red","green","black"/)
    resB_0@xyDashPatterns         = (/0,0,0,0/)
    resB_0@xyLineThicknessF       = 3
    resB_0@trYMinF                = -2.5            
    resB_0@trYMaxF                = 2.5
    resB_0@trXMinF                = -35
    resB_0@trXMaxF                = 65

    resB_0@pmLegendDisplayMode    = "Always"
    resB_0@pmLegendOrthogonalPosF = -0.5
    resB_0@pmLegendParallelPosF   = 0.89
    resB_0@pmLegendWidthF         = 0.13
    resB_0@pmLegendHeightF        = 0.1
    resB_0@xyExplicitLabels       = (/"Indian","Atlantic","Pacific","Total"/)
plotsB(1)                          = gsn_csm_xy(wksB,lat,y2, resB_0)

resB_1    = True
    resB_1@gsnDraw                = False
    resB_1@gsnFrame               = False           
    resB_1@vpHeightF              = 0.37
    resB_1@vpWidthF               = 0.6
    resB_1@tiXAxisString          = "Latitude" 
    resB_1@tiYAxisString          = "Net Surface Heat Flux" 
    resB_1@gsnYRefLine            = 0
    resB_1@gsnYRefLineThicknessF  = 1.5
    resB_1@xyLineColors           = (/"purple","red","green","black"/)
    resB_1@xyDashPatterns         = (/0,0,0,0/)
    resB_1@xyLineThicknessF       = 3
;    resB_1@trYMinF                = -2.5            
;    resB_1@trYMaxF                = 2.5
    resB_1@trXMinF                = -35
    resB_1@trXMaxF                = 65

    resB_1@pmLegendDisplayMode    = "Always"
    resB_1@pmLegendOrthogonalPosF = -0.5
    resB_1@pmLegendParallelPosF   = 0.89
    resB_1@pmLegendWidthF         = 0.13
    resB_1@pmLegendHeightF        = 0.1
    resB_1@xyExplicitLabels       = (/"Indian","Atlantic","Pacific","Total"/)
plotsB(0)                          = gsn_csm_xy(wksB,lat,y1, resB_1)

resBP                              = True
gsn_panel(wksB, plotsB, (/1,2/), resBP)

![heat_lat](../photo/heat_lat.png )